In [2]:
import gym

env = gym.make('Taxi-v3').env
env.render()


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
print("Action space {}".format(env.action_space))
print("State space {}".format(env.observation_space))

      

Action space Discrete(6)
State space Discrete(500)


In [5]:
state = env.encode(3, 1, 2, 0)
print('State:' , state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [7]:
env.s = 328

epoches = 0
penalties, reward = 0, 0

frames = []

done = False

while not done:
    action = env.action_space.sample()
    state , reward, done, info = env.step(action)
    
    if reward == -10:
        penalties +=1
        
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    })
    epoches += 1

print("Timesteps to solve{}".format(epoches))
print("Penalties incurred {}".format(penalties))
print(frames[-1]['frame'])

Timesteps to solve2538
Penalties incurred 835
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



In [8]:
from IPython.display import clear_output
from time import sleep


def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"timestep: {i + 1} ")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(0.05)
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

timestep: 931 
State: 0
Action: 5
Reward: 20


In [10]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
print(q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [11]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output
from time import sleep


# Hyperparameters
alpha = 0.85
gamma = 0.70
epsilon = 0.12

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()
    epsilon = 0.11 - ((i) / 1000000)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
            print(action)
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        sleep(0.001)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 5000
0
3
1
3
0
0
4
5
2
2
4
2
0
5
5
3
0
3


KeyboardInterrupt: 

In [51]:
print(epsilon)
print(q_table[348])
env.s = 348
env.render()

0.009999999999999995
[ -3.01037258  -2.67422442  -2.87195709  -2.67422442 -11.87196343
 -11.8719571 ]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


In [54]:

total_epochs, total_penalties = 0, 0
episodes = 100

frames = []


for i in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        frames.append(
        {
            'frame': env.render(mode='ansi'),
            'action': action,
            'state': state,
            'reward': reward
            
        }
        )
        if reward == -10:
            penalties += 1

        epochs += 1
    if i % 10 == 0:
        print_frames(frames)
        print('episode:', i)
        sleep(1)
    frames = []
    total_penalties += penalties
    total_epochs += epochs

    
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"timesteps {i+1}")
        print(f"Action {frame['action']}")
        print(f"State {frame['state']}")
        print(f"Reward {frame['reward']}")
        sleep(1)

    
print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

timesteps 17
Action 5
State 475
Reward 20
episode: 90
Results after 100 episodes:
Average timesteps per episode: 13.13
Average penalties per episode: 0.0
